#  Problem

Banglore House Price Data to find the approximate prize for the house

In [ ]:
# Importing Liberaries 
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams["figure.figsize"] = (20,10)

In [ ]:
# load Data Csv file in dataframe
df = pd.read_csv('../input/bengaluru-house-price-data/Bengaluru_House_Data.csv')
df.head()

Data Analysis

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df['area_type'].unique()

In [ ]:
df['area_type'].value_counts()

In [ ]:
plt.figure(figsize=(12,3))
df.area_type.value_counts().plot(kind='bar')

In [ ]:
plt.figure(figsize=(12,3))
df.bath.value_counts().plot(kind='bar')

In [ ]:
plt.figure(figsize=(12,3))
df.balcony.value_counts().plot(kind='bar')

In [ ]:
#  drop some columns who are not usefull for build our model
df1 = df.drop(['balcony','area_type','society','availability'],axis='columns')
df1.head()

In [ ]:
df1.isnull().sum()

In [ ]:
df2 = df1.dropna()
df1.shape

In [ ]:
df2.isnull().sum()

In [ ]:
df2['size'].unique() # here bhk and bedrooms both are same 

Feature Engineering

In [ ]:
df2['New_Size']=df2['size'].apply(lambda i: int(i.split(' ')[0]))
df2.head()

In [ ]:
df2.New_Size.unique()

In [ ]:
sns.catplot(x='New_Size',y='bath',data=df2,kind='bar',alpha=.6, height=6,ci="sd");

In [ ]:
df2.total_sqft.unique()  # 1133-1384 is a range 

In [ ]:
# for convert all total_sqft features data in range

def convertFloat(i):
    try:
        float(i)
    except:
        return False
    return True

df2[~df2['total_sqft'].apply(convertFloat)].head(10)

In [ ]:
df2[~df2['total_sqft'].apply(convertFloat)].tail(10)

In [ ]:
def convertInInt(i):
    tokens = i.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(i)
    except:
        return None
    
df2['total_sqft']= df['total_sqft'].apply(convertInInt)
df2.head()

In [ ]:
df2.loc[12560]  # total_sqft is null in 12560, 12652 and many more

In [ ]:
df2.isnull().sum()

In [ ]:
df2.shape

In [ ]:
df3 = df2.dropna()
df3.shape

In [ ]:
df3.head(2)

In [ ]:
# we don't know what is the price of a square feet so make new feature that's call price_per_sqft
df3['price_per_sqft'] = df3['price']*100000/df3['total_sqft']
df3.head()

In [ ]:
df4 = df3.copy()
df4['price_per_sqft'].describe()

In [ ]:
df4.shape

In [ ]:
len(df4.location.unique())

In [ ]:
df4.location.unique() # our location feature in to much unique values 

In [ ]:
df4.location = df4.location.apply(lambda i: i.strip()) 
locationS = df4['location'].value_counts(ascending=False)
locationS

In [ ]:
locationS.values.sum()

In [ ]:
len(locationS[locationS>10])

In [ ]:
len(locationS[locationS<10])

In [ ]:
len(locationS[locationS<=10])

In [ ]:
location_less_10 = locationS[locationS<=10]
location_less_10

In [ ]:
df4.location = df4.location.apply(lambda i : 'other' if i in location_less_10 else i)
len(df4.location.unique())  # Hurrah!

In [ ]:
# now we want to remove some outliers 
df4[df4.total_sqft/df4.New_Size<300].head()
# now you can see in dataframe that 1020 sqft with 6 bedroom....My home is 900 sqft with 2 bedroom and thats both room are medium size
# and another 600 sqft with 8 rooms and 9 bath..... haha its may be all baths no one room in home
# so its all are conduct as a data error or outliers

In [ ]:
print(len(df4[df4.total_sqft/df4.New_Size<300]))

In [ ]:
df5 = df4[(df4.total_sqft/df4.New_Size<300)]
df5.shape

In [ ]:
df5 = df4[~(df4.total_sqft/df4.New_Size<300)]
df5.shape

In [ ]:
df5.price_per_sqft.describe()

In [ ]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df6 = remove_pps_outliers(df5)
df6.shape

In [ ]:
plt.figure(figsize=(20,10))
features = df6.columns
for i, j in enumerate(features):
    plt.subplot(3,3,i+1)
    plt.title(f'Histogram of {j}', fontsize= 15)
    bins = len(df6[j].unique())
    plt.hist(df6[j], bins=bins,rwidth=0.8,linewidth=2, edgecolor= 'y')
plt.subplots_adjust(hspace=0.5)

In [ ]:
df6.New_Size.unique()

In [ ]:
def plot_scatter(df,location):
    bhk2 = df[(df.location==location) & (df.New_Size==2)]
    bhk3 = df[(df.location==location)&(df.New_Size==3)]
    bhk4 = df[(df.location==location)&(df.New_Size==4)]
    matplotlib.rcParams['figure.figsize']=(10,8)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK', s=50, marker = '.')
    plt.scatter(bhk3.total_sqft,bhk3.price,color='green',label='3 BHK', s=50, marker = '*')
    plt.scatter(bhk4.total_sqft,bhk4.price,color='red',label='4 BHK', s=50, marker = '+')
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(location)
    plt.legend()

plot_scatter(df6,'Whitefield')

In [ ]:
plot_scatter(df6,'Thanisandra')

In [ ]:
def remove_outlier_bed(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('New_Size'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('New_Size'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

df7 = remove_outlier_bed(df6)
df7.shape

In [ ]:
plot_scatter(df7,'Thanisandra')  # previous graph vs new graph can easily understand our outliers are removed

In [ ]:
matplotlib.rcParams["figure.figsize"] = (10,4)
plt.hist(df7.price_per_sqft,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

In [ ]:
df7.bath.unique()

In [ ]:
df7[df7.bath>10]

In [ ]:
df7[df7.bath>df7.New_Size+2]

In [ ]:
df7.shape

In [ ]:
df8 = df7[df7.bath<df7.New_Size+2]
df8.shape

In [ ]:
df8.head(2)

In [ ]:
df9 = df8.drop(['size','price_per_sqft'], axis= 1)
df9.head(1)

In [ ]:
# one hot encoding for locatin
dummies = pd.get_dummies(df9.location)
dummies.head(2)

In [ ]:
df10 = pd.concat([df9,dummies],axis=1)
df10 = df10.drop(['other','location'],axis=1)
df10.head(2)

# Model Building

In [ ]:
x = df10.drop(['price'], axis=1)
y = df10.price
x.head(2)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train,y_train)
lr.score(x_test,y_test)

In [ ]:
# use of k fold cross validation 
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, random_state=10, test_size=0.2)
cross_val_score(LinearRegression(),x,y,cv=cv)

GridSearch cv to find best model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB



In [ ]:
def bestModel(x,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(x,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

bestModel(x,y)

  Winner is LinearRegression with parameter Normalize = False

In [ ]:
# our first model give us best score 
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train,y_train)
lr.score(x_test,y_test)

In [ ]:
# for prediction
x.columns

In [ ]:
type(np.where(x.columns=='bath'))

In [ ]:
type(np.where(x.columns=='bath')[0])

In [ ]:
type(np.where(x.columns=='bath')[0][0])

In [ ]:
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(x.columns==location)[0][0]

    X = np.zeros(len(x.columns))
    X[0] = sqft
    X[1] = bath
    X[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return lr.predict([X])[0]

In [ ]:
predict_price('Vittasandra',1000, 2, 2)

In [ ]:
import pickle
with open('banglore_home_prices_prediction_model.pickle','wb') as f:
    pickle.dump(lr,f)